In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
import warnings
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pandas import Series
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import StandardScaler
from math import sqrt
from datetime import datetime,timedelta
from dateutil.rrule import rrule, HOURLY

warnings.filterwarnings("ignore")

frames=[]
dffinal=pd.read_csv("cleaned.csv", index_col='date',parse_dates=True)

dffinal=dffinal.drop(columns=['NO','CH4','BEN','CO','EBE','MXY','NMHC','NOx','OXY','O_3','PM10','PXY','SO_2','TCH','TOL','PM25'])

print(dffinal.head())


                          NO_2     station
date                                      
2001-01-01 01:00:00  59.150002  28079018.0
2001-01-01 02:00:00  55.250000  28079018.0
2001-01-01 03:00:00  54.740002  28079018.0
2001-01-01 04:00:00  48.160000  28079018.0
2001-01-01 05:00:00  43.700001  28079018.0


In [2]:
split_start_train = pd.Timestamp('2002-01-01')
split_end_train = split_start_test = pd.Timestamp('2012-01-01')
split_end_test= split_start_valid = pd.Timestamp('2013-01-01')
split_end_valid = pd.Timestamp('2014-01-01');
df_train = dffinal.loc[split_start_train:split_end_train]
df_test = dffinal.loc[split_start_test:split_end_test]
df_valid =  dffinal.loc[split_start_valid:split_end_valid]


In [3]:
# Define the p, d and q parameters to take any value between 0 and 2
import itertools
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
pdqs = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
print(df_train.head())

                           NO_2     station
date                                       
2002-01-01 00:00:00   90.430000  28079018.0
2002-01-01 01:00:00   44.000000  28079018.0
2002-01-01 02:00:00   44.000000  28079018.0
2002-01-01 03:00:00  107.300003  28079018.0
2002-01-01 04:00:00   82.419998  28079018.0


In [8]:
#determine what the best model would be using a for loop
ans = []
for comb in pdq:
    for combs in pdqs:
        try:
            mod = sm.tsa.statespace.SARIMAX(df_train,
                                            order=comb,
                                            seasonal_order=combs,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            output = mod.fit()
            ans.append([comb, combs, output.aic])
            print('ARIMA {} x {}12 : AIC Calculated ={}'.format(comb, combs, output.aic))
        except:
            continue

In [6]:
#print out the best model parameters
ans_df = pd.DataFrame(ans, columns=['pdq', 'pdqs', 'aic'])
print(ans_df.loc[ans_df['aic'].idxmin()])

TypeError: reduction operation 'argmin' not allowed for this dtype

In [7]:
#Use pdq and pdqs above to fit the data with the ARIMA model
ARIMA_MODEL = sm.tsa.statespace.SARIMAX(df_train,
                                order=(1, 0, 1),
                                seasonal_order=(0, 1, 1, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)

output_df_train = ARIMA_MODEL.fit()

print(output_df_train.summary().tables[1])

ValueError: SARIMAX models require univariate `endog`. Got shape (86617, 2).

In [1]:
#Use plot_diagnostics with results calculated above to make sure 
#our assumptions of normality and correlation hold
output_df_train.plot_diagnostics(figsize=(15, 18))
plt.show()

NameError: name 'output_df_train' is not defined